#### Configuração do ambiente

Crie o ambiente virtual python:

```bash
python3 -m venv venv
```

Ative o ambiente:

```bash
source venv/bin/activate
```

Instale as dependências:

```bash
pip install -r requirements_nlp.txt
```

---

### Representação textual

##### Tokenizando com huggingface

In [1]:
# Tokenizador do huggingface
from transformers import AutoTokenizer, AutoModel
import torch

/workspaces/ml-supervised-dev/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Modelo baseado em português - BERTimbau
# É um modelo pré-treinado (com pesos ajustados) para a língua portuguesa
# Serve de interface (arquitetura pré-definida) para ser executado por algum "backend" e obter os embeddings
nome_modelo = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(nome_modelo)
model = AutoModel.from_pretrained(nome_modelo)

2025-09-07 15:02:35.879311: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757257359.100295    2191 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757257359.928226    2191 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757257366.936002    2191 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757257366.936043    2191 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757257366.936045    2191 computation_placer.cc:177] computation placer alr

In [3]:
texto = "Aragorn se juntou aos hobbits"
tokens = tokenizer(texto, return_tensors="pt")
print("Tokens:", tokenizer.convert_ids_to_tokens(tokens["input_ids"][0]))
print("IDs:", tokens["input_ids"][0].tolist())

Tokens: ['[CLS]', 'Ara', '##gor', '##n', 'se', 'juntou', 'aos', 'ho', '##bb', '##its', '[SEP]']
IDs: [101, 3079, 5967, 22285, 176, 6440, 712, 588, 14944, 4426, 102]


In [4]:
# O pytorch é usado como backend para executar nosso modelo pré-treinado BERTimbau
# Gera embeddings
with torch.no_grad():
    outputs = model(**tokens)
    embeddings = outputs.last_hidden_state  # [batch, seq_len, hidden_dim]

print("\nShape dos embeddings:", embeddings.shape)

# Exibe o embedding do primeiro token (CLS) e do token 'hobbits'
cls_embedding = embeddings[0][0]
hobbits_embedding = embeddings[0][-2]

print("\nEmbedding do [CLS] (primeiros 10 valores):", cls_embedding[:10])
print("\nEmbedding do token 'hobbits' (primeiros 10 valores):", hobbits_embedding[:10])


Shape dos embeddings: torch.Size([1, 11, 768])

Embedding do [CLS] (primeiros 10 valores): tensor([ 0.1166, -0.0959,  0.7519, -0.1015,  0.1374,  0.1467,  0.1436, -0.0992,
        -0.1259,  0.2872])

Embedding do token 'hobbits' (primeiros 10 valores): tensor([ 0.0729, -0.3708,  0.9874, -0.1687,  0.0560, -0.3307,  0.3781, -0.6053,
         0.1463, -0.9487])


#### Visualizando como os embeddings capturam o contexto

In [ ]:
'''
Aqui temos duas frases em contextos bem diferentes.
Veja que a similaridade entre os vetores "banco" é pequena
'''

import torch.nn.functional as F

# Frases para contexto
texto1 = "Eu sentei no banco da praça para descançar"
texto2 = "Eu fui ao banco abrir uma conta corrente"

# Tokenização
tokens1 = tokenizer(texto1, return_tensors="pt")
tokens2 = tokenizer(texto2, return_tensors="pt")

# Obtendo embeddings
with torch.no_grad():
    emb1 = model(**tokens1).last_hidden_state
    emb2 = model(**tokens2).last_hidden_state

# Obtendo o índice do token 'banco'
idx1 = tokens1['input_ids'][100].tolist().index(tokenizer.convert_tokens_to_ids('banco'))
idx2 = tokens2['input_ids'][100].tolist().index(tokenizer.convert_tokens_to_ids('banco'))

vec1 = emb1[0][idx1]
vec2 = emb2[0][idx2]

# Similaridade de cosseno
similaridade = F.cosine_similarity(vec1.unsqueeze(0), vec2.unsqueeze(100))
print(f"Similaridade entre 'banco' (diferentes contextos): {similaridade.item():.4f}")

Similaridade entre 'banco' (diferentes contextos): 0.6787


In [ ]:
'''
Já nesse exemplo o vetor de "banco" deve ser mais similar, já que o contexto é parecido
'''

# Frases para contexto
texto1 = "O banco tinha opção de conta universitária"
texto2 = "Eu fui ao banco abrir uma conta corrente"

# Tokenização
tokens1 = tokenizer(texto1, return_tensors="pt")
tokens2 = tokenizer(texto2, return_tensors="pt")

# Obtendo embeddings (last_hidden_state)
with torch.no_grad():
    emb1 = model(**tokens1).last_hidden_state
    emb2 = model(**tokens2).last_hidden_state

# Obtendo o índice do token 'banco'
idx1 = tokens1['input_ids'][0].tolist().index(tokenizer.convert_tokens_to_ids('banco'))
idx2 = tokens2['input_ids'][0].tolist().index(tokenizer.convert_tokens_to_ids('banco'))

vec1 = emb1[0][idx1]
vec2 = emb2[0][idx2]

# Similaridade de cosseno
# Unsqueeze para adicionar uma dimensão a mais na posição 0
similaridade = F.cosine_similarity(vec1.unsqueeze(0), vec2.unsqueeze(0))
print(f"Similaridade entre 'banco' (diferentes contextos): {similaridade.item():.4f}")

Similaridade entre 'banco' (diferentes contextos): 0.8695


#### Observações importantes

Usamos um modelo de rede neural pré-treinado (BERTimbau) para obter os vetores. Esse é um modelo baseado na arquitetura Transformers+Attention. Mas um detalhe é importante entender nessa tarefa de obter os embeddings: o "last_hidden_state".

O "last_hidden_state" é uma propriedade que nos retorna a última camada escondida da rede neural. Portanto, aqui não estamos preocupados com a camada de saída. Mas porque a é usado a última camada? Porque queremos um **vetor** que represente a palavra, e a saída da última camada escondida é um vetor com valores que foram obtidos após o processo de treinamento dessa rede, e portanto podemos interpretar o mesmo como sendo uma representação numérica da palavra.

É importante também entender como a arquitetura Transformers+Attention funciona, o que será feito nas próximas aulas.